## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-22-10-58-35 +0000,bbc,UK holds off joining Trump's Board of Peace ov...,https://www.bbc.com/news/articles/cp9jj1j74ggo...
1,2026-01-22-10-57-00 +0000,nyt,Live Updates: Trump Hosts ‘Board of Peace’ Sig...,https://www.nytimes.com/live/2026/01/22/us/tru...
2,2026-01-22-10-52-51 +0000,nyt,"Friedrich Merz, German Chancellor, Urges Europ...",https://www.nytimes.com/2026/01/22/us/politics...
3,2026-01-22-10-52-00 +0000,wsj,Auto & Transport Roundup: Market Talk,https://www.wsj.com/business/auto-transport-ro...
4,2026-01-22-10-51-47 +0000,nyt,Three Dead After Shooting in Rural Australian ...,https://www.nytimes.com/2026/01/22/world/austr...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2348/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
4,trump,101
285,greenland,33
14,davos,33
573,court,17
43,deal,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
378,2026-01-21-13-20-25 +0000,cbc,Trump calls Denmark 'ungrateful' — but tells D...,https://www.cbc.ca/news/world/livestory/trump-...,217
234,2026-01-21-20-46-06 +0000,cbc,"Trump reiterates desire to acquire Greenland, ...",https://www.cbc.ca/news/world/trump-davos-spee...,209
74,2026-01-22-07-39-58 +0000,bbc,Trump says 'framework of a future deal' discus...,https://www.bbc.com/news/articles/cgezx40r7d7o...,201
275,2026-01-21-19-14-45 +0000,nyt,Did the U.S. Ever Own Greenland? Fact-Checking...,https://www.nytimes.com/2026/01/21/us/politics...,190
325,2026-01-21-16-58-00 +0000,wsj,EU Lawmakers Put Trade Deal With U.S. on Ice A...,https://www.wsj.com/world/europe/eu-lawmakers-...,186


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
378,217,2026-01-21-13-20-25 +0000,cbc,Trump calls Denmark 'ungrateful' — but tells D...,https://www.cbc.ca/news/world/livestory/trump-...
174,87,2026-01-21-23-21-00 +0000,wsj,Some Democrats join Republicans on the House O...,https://www.wsj.com/politics/republicans-turn-...
281,70,2026-01-21-19-00-40 +0000,latimes,Supreme Court wary of Trump's bid to fire Fed ...,https://www.latimes.com/politics/story/2026-01...
123,59,2026-01-22-02-17-39 +0000,cbc,Former Uvalde school police officer found not ...,https://www.cbc.ca/news/world/uvalde-police-sc...
74,53,2026-01-22-07-39-58 +0000,bbc,Trump says 'framework of a future deal' discus...,https://www.bbc.com/news/articles/cgezx40r7d7o...
284,46,2026-01-21-18-54-48 +0000,nypost,Barron Trump ‘saved’ friend’s life by calling ...,https://nypost.com/2026/01/21/us-news/barron-t...
62,43,2026-01-22-08-43-21 +0000,wsj,"Stock Market Today: Dow Futures, Global Stocks...",https://www.wsj.com/livecoverage/stock-market-...
102,42,2026-01-22-03-35-36 +0000,nyt,ICE Said Agents Can Enter Homes Without Judici...,https://www.nytimes.com/2026/01/21/us/politics...
70,41,2026-01-22-07-57-22 +0000,bbc,Three dead in shooting in Australia's New Sout...,https://www.bbc.com/news/articles/c4g00mv29j5o...
301,41,2026-01-21-18-09-21 +0000,nyt,Trump Calls Out Mark Carney in Davos Speech an...,https://www.nytimes.com/2026/01/21/us/politics...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
